In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

# Inicializa o driver
driver = webdriver.Chrome()
driver.get("https://sa.sinnc.app/conims/5/IDSSaude.dll")

# Injetar o script de monitoramento
script = """
(function() {
    window.actions = [];

    document.addEventListener('click', function(event) {
        let path = event.composedPath();
        let element = path[0];
        let selector = element.tagName.toLowerCase();

        if (element.id) {
            selector += `#${element.id}`;
        } else if (element.className) {
            selector += `.${element.className.split(' ').join('.')}`;
        }

        window.actions.push({
            type: 'click',
            selector: selector,
            timestamp: Date.now()
        });
    });

    document.addEventListener('input', function(event) {
        let path = event.composedPath();
        let element = path[0];
        let selector = element.tagName.toLowerCase();

        if (element.id) {
            selector += `#${element.id}`;
        } else if (element.className) {
            selector += `.${element.className.split(' ').join('.')}`;
        }

        window.actions.push({
            type: 'input',
            selector: selector,
            value: element.value,
            timestamp: Date.now()
        });
    });

    document.addEventListener('keydown', function(event) {
        if (event.key === 'Escape') {
            // Retorna as ações gravadas para o Python
            return JSON.stringify(window.actions);
        }
    });
})();
"""
driver.execute_script(script)

print("Script injetado. Realize ações no navegador e pressione ESC para finalizar.")

# Aguardar até que ESC seja pressionado
input("Pressione Enter após pressionar ESC no navegador para finalizar...")

# Recuperar as ações registradas
actions = driver.execute_script("return JSON.stringify(window.actions);")

# Salvar no arquivo JSON
with open("actions.json", "w") as file:
    file.write(actions)

print("Ações registradas salvas em 'actions.json'")


Script injetado. Realize ações no navegador e pressione ESC para finalizar.
Ações registradas salvas em 'actions.json'
